In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
from __future__ import print_function 
from __future__ import division

try:
    import torch
    import torch.nn as nn
    import torch.optim as optim
    import numpy as np
    import torchvision
    import matplotlib.pyplot as plt
    import time
    import os
    import copy
    from torchvision import datasets, models, transforms
    print("PyTorch Version: ",torch.__version__)
    print("Torchvision Version: ",torchvision.__version__)
except Exception as error:
    !pip install {str(error)[17:-1]}
except:
    print("Pacote não encontrado")

PyTorch Version:  1.7.1+cu110
Torchvision Version:  0.8.2+cu110


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
import torchvision
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = torchvision.models.resnet152().to(device)
try:
    from torchsummary import summary
except Exception as error:
    !pip install {str(error)[17:-1]}
except:
    print("Pacote não encontrado")
summary(model, (3, 224, 224))

 14]           2,048
            ReLU-209         [-1, 1024, 14, 14]               0
      Bottleneck-210         [-1, 1024, 14, 14]               0
          Conv2d-211          [-1, 256, 14, 14]         262,144
     BatchNorm2d-212          [-1, 256, 14, 14]             512
            ReLU-213          [-1, 256, 14, 14]               0
          Conv2d-214          [-1, 256, 14, 14]         589,824
     BatchNorm2d-215          [-1, 256, 14, 14]             512
            ReLU-216          [-1, 256, 14, 14]               0
          Conv2d-217         [-1, 1024, 14, 14]         262,144
     BatchNorm2d-218         [-1, 1024, 14, 14]           2,048
            ReLU-219         [-1, 1024, 14, 14]               0
      Bottleneck-220         [-1, 1024, 14, 14]               0
          Conv2d-221          [-1, 256, 14, 14]         262,144
     BatchNorm2d-222          [-1, 256, 14, 14]             512
            ReLU-223          [-1, 256, 14, 14]               0
          Conv2d-22

In [4]:
def get_free_memory():
    import GPUtil
    CUDA_VISIBLE_DEVICES = os.environ.get('CUDA_VISIBLE_DEVICES')
    memory = 0
    for GPU in GPUtil.getGPUs():
        if CUDA_VISIBLE_DEVICES is None or str(GPU.id) in CUDA_VISIBLE_DEVICES: 
            memory += GPU.memoryFree
    return memory


def no_free_mem( mem_per_sample, available ):
    return 5*np.array(mem_per_sample).max() > available

def main():
    model = torchvision.models.resnet152()
    model = model.cuda()

    if torch.cuda.device_count() > 1:
        model = torch.nn.DataParallel(model)

    max_len = 60000

    x = get_first_sample(X, max_len, model.inputs_cfg)
    y = get_first_sample(Y, max_len, model.outputs_cfg, output=True)
    
    optimizer = optim.Adam(filter(lambda param: param.requires_grad, model.parameters()))

    moremem = True
    batch_size = 1
    prev_freemem = get_free_memory()
    mem_per_sample = [0]
    optimizer.zero_grad()
    
    while moremem:
        y_pred, _, _ = model(x)
        freemem = get_free_memory()
        if no_free_mem( mem_per_sample, freemem ): break
        
        loss, _ = model.loss(y_pred, y)
        
        freemem = min(freemem, get_free_memory())
        if no_free_mem( mem_per_sample, freemem ): break

        loss.backward()
        freemem = min(freemem, get_free_memory())
        if no_free_mem( mem_per_sample, freemem ): break

        optimizer.step()
        freemem = min(freemem, get_free_memory())
        if no_free_mem( mem_per_sample, freemem ): break

        if prev_freemem - freemem > 0:
            mem_per_sample.append(prev_freemem - freemem)

        if no_free_mem( mem_per_sample, freemem ): break
    
        batch_size += 1
        prev_freemem = min(prev_freemem, freemem)

        x = insert_sample(x)
        y = insert_sample(y)
        
    print("GUESSING batch_size, ", batch_size)

In [5]:
main()

NameError: name 'get_first_sample' is not defined

In [17]:

try:
    from pytorch_lightning import Trainer
except Exception as error:
    !pip install {str(error)[17:-1]}
except:
    print("Pacote não encontrado")

     |████████████████████████████████| 829 kB 1.3 MB/s 
     |████████████████████████████████| 829 kB 8.7 MB/s 
     |████████████████████████████████| 103 kB 11.2 MB/s 
     |████████████████████████████████| 1.5 MB 9.9 MB/s 
     |████████████████████████████████| 159 kB 13.6 MB/s 
     |████████████████████████████████| 324 kB 12.9 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491059 sha256=a28ea53bee8d965dfc2c61007944d276eab67b4a7b4ccc54f50ed6b4b40f55f9
  Stored in directory: /home/matheuscosta/.cache/pip/wheels/8e/70/28/3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4
Successfully built future
ERROR: awsebcli 3.19.3 has requirement future<0.17.0,>=0.16.0, but you'll have future 0.18.2 which is incompatible.
  Attempting uninstall: future
    Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0


In [18]:
from pytorch_lightning import Trainer

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = torchvision.models.resnet152().to(device)
trainer = Trainer(auto_scale_batch_size='binsearch')
trainer.tune(model)

GPU available: True, used: False
TPU available: None, using: 0 TPU cores


ModuleAttributeError: 'ResNet' object has no attribute 'automatic_optimization'